In [ ]:
import pandas as pd
import numpy as np
import graph_tool.all as gt
import sys
sys.path.append("/home/jovyan/work/phd/hSBM_Topicmodel/")
sys.path.append("/home/jovyan/work/phd/trisbm/")

from sbmtm import sbmtm
from trisbm import trisbm
import scanpy as sc

In [ ]:
df = pd.read_csv("mainTable_fpkm.csv", index_col=0)

In [ ]:
df_mirna = pd.read_csv("mainTable_mirna.csv", index_col=0)

In [ ]:
df_files = pd.read_csv("files.dat")

In [ ]:
df = df[df.columns[df.columns.isin(df_files["file_fpkm"])]]
df_mirna = df_mirna[df_mirna.columns[df_mirna.columns.isin(df_files["file_mirna"])]]

In [ ]:
df.columns=df_files.set_index("file_fpkm").reindex(index=df.columns)["cases.0.submitter_id"]
df_mirna.columns=df_files.set_index("file_mirna").reindex(index=df_mirna.columns)["cases.0.submitter_id"]
df_mirna = df_mirna.reindex(columns=df.columns)

In [ ]:
df_files.set_index("cases.0.submitter_id", inplace=True)

## HVG

In [ ]:
adata = sc.AnnData(
    X=df.transpose().reindex(index=df_files.index), 
    obs=df_files)

In [ ]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000)
sc.pl.highly_variable_genes(adata)

In [ ]:
hvg = adata.var[adata.var["highly_variable"]==True].index

## Highly variable miRNA

In [ ]:
adata = sc.AnnData(
    X=df_mirna.transpose().reindex(index=df_files.index), 
    obs=df_files)

In [ ]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=1000)
sc.pl.highly_variable_genes(adata)

In [ ]:
hvmiRNA = adata.var[adata.var["highly_variable"]==True].index.unique()

# hSBM

In [ ]:
hsbm = sbmtm()

In [ ]:
hsbm.make_graph_from_BoW_df(df.reindex(index=hvg))
hsbm.g

In [ ]:
hsbm.save_graph("graph_sbmtm.xml.gz")

## triSBM

In [ ]:
df_all = df.reindex(index=hvg).append(df_mirna.reindex(index=hvmiRNA))

In [ ]:
trisbm_model = trisbm()
trisbm_model.make_graph(df_all, lambda gene: 1 if "ENSG" in gene else 2)

In [ ]:
trisbm_model.g

In [ ]:
trisbm_model.save_graph("graph_trisbm.xml.gz")